字符串输出解析器 StrOutputParser

In [3]:
# 1、获取大模型
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser, XMLOutputParser

import os
import dotenv
from langchain_core.utils import pre_init
from langchain_openai import ChatOpenAI

dotenv.load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("DASHSCOPE_API_KEY")
os.environ["OPENAI_BASE_URL"] = os.getenv("DASHSCOPE_BASE_URL")


########核心代码############
chat_model = ChatOpenAI(model="qwen-turbo",
                        temperature=0.7,    # 1=创造力
                        max_tokens=100,)     # 最大数量

# 2、调用大模型
response = chat_model.invoke("什么是大语言模型？")
# print(type(response))   #AIMessage

#3、如何获取一个字符串的输出结果呢？
# 方式1：自己调用输出结果的content
# print(response.content)

# 方式2：使用StrOutputParser
parser = StrOutputParser()
str_response = parser.invoke(response)
print(type(str_response))  #<class 'str'>
print(str_response)


<class 'str'>
大语言模型（Large Language Model，简称 LLM）是一种基于深度学习的自然语言处理（NLP）模型，它通过在大量文本数据上进行训练，能够理解和生成人类语言。这些模型通常包含数十亿甚至数千亿个参数，因此被称为“大”语言模型。

### 大语言模型的核心特点包括：

1. **大规模参数**：  
   大语言模型通常具有庞大的参数量，这使得它们能够捕捉更复杂的语言模式和语义关系。

2. **预训练与微调**：  
   这些模型通常先在大量文本数据上进行**预训练**，学习语言的通用表示；然后在特定任务上进行**微调**，以适应具体的应用场景，如问答、文本生成、翻译等。

3. **强大的语言理解能力**：  
   大语言模型可以理解上下文、识别实体、推理逻辑，并能生成连贯、自然的文本。

4. **多任务处理能力**：  
   一个大语言模型可以执行多种任务，例如：
   - 文本生成（如写故事、写代码）
   - 问答系统
   - 翻译
   - 情感分析
   - 对话系统（如聊天机器人）

5. **零样本/少样本学习能力**：  
   即使没有经过特定任务的微调，大语言模型也能在少量示例的情况下完成新任务。

---

### 常见的大语言模型包括：

- **GPT系列**（如 GPT-3、GPT-4）——由 OpenAI 开发。
- **BERT**、**RoBERTa**、**ALBERT** —— 由 Google 开发的预训练模型。
- **T5**、**BART** —— 由 Google 和 Facebook 开发的序列到序列模型。
- **LLaMA**、**ChatGLM**、**Qwen（通义千问）** —— 由 Meta、智普清言、阿里巴巴等公司开发的大规模语言模型。

---

### 应用场景

大语言模型广泛应用于以下领域：

- **智能客服**：自动回答用户问题。
- **内容创作**：生成文章、故事、剧本等。
- **代码生成与理解**：辅助程序员编写或理解代码。
- **教育**：个性化学习、答疑解惑。
- **数据分析**：从文本中提取信息、生成报告。
- **虚拟助手**：如 Siri、Alexa、小爱同学等。

---

### 总结

**大语言模型**是人工智

JsonOutputParser : Json输出解析器

In [6]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate

chat_model = ChatOpenAI(model="qwen-turbo",
                        temperature=0.7,    # 1=创造力
                        max_tokens=100,)     # 最大数量

chat_prompt_template = ChatPromptTemplate.from_messages([
    ("system", "你是一个靠谱的{role}"),
    ("human", "{question}")
])

# 正确的：
prompt = chat_prompt_template.invoke(
    input={"role": "人工智能专家", "question": "人工智能用英文怎么说？问题用q表示，答案用a表示，返回一个JSON格式的数据"})

# 错误的：
# prompt = chat_prompt_template.invoke(input={"role":"人工智能专家","question":"人工智能用英文怎么说？"})

response = chat_model.invoke(prompt)
print(response.content)

# 获取一个JsonOutputParser的实例
parser = JsonOutputParser()

json_result = parser.invoke(response)
print(json_result)

{
  "Q": "What is the English word for artificial intelligence?",
  "A": "Artificial intelligence is translated into English as 'artificial intelligence'."
}
{'Q': 'What is the English word for artificial intelligence?', 'A': "Artificial intelligence is translated into English as 'artificial intelligence'."}


方式2：

In [7]:
parser = JsonOutputParser()

print(parser.get_format_instructions())

Return a JSON object.


In [8]:
# 引入依赖包
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate

# 初始化语言模型
chat_model = ChatOpenAI(model="qwen-turbo",
                        temperature=0.7,    # 1=创造力
                        max_tokens=100,)     # 最大数量

joke_query = "告诉我一个笑话。"

# 定义Json解析器
parser = JsonOutputParser()

#以PromptTemplate为例
prompt_template = PromptTemplate.from_template(
    template="回答用户的查询\n 满足的格式为{format_instructions}\n 问题为{question}\n",
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

prompt = prompt_template.invoke(input={"question": joke_query})
response = chat_model.invoke(prompt)
print(response)

json_result = parser.invoke(response)
print(json_result)

content='{\n  "response": "为什么数学书总是很忧郁？\\n因为它有太多的问题。"\n}' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 35, 'total_tokens': 59, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'qwen-turbo', 'system_fingerprint': None, 'id': 'chatcmpl-280f23b4-960f-4aa5-856c-d7641438675b', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None} id='run--38dee115-745c-4da8-91a1-b76507f16f1f-0' usage_metadata={'input_tokens': 35, 'output_tokens': 24, 'total_tokens': 59, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}}
{'response': '为什么数学书总是很忧郁？\n因为它有太多的问题。'}


In [9]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate

chat_model = ChatOpenAI(model="qwen-turbo",
                        temperature=0.7,    # 1=创造力
                        max_tokens=100,)     # 最大数量

chat_prompt_template = ChatPromptTemplate.from_messages([
    ("system", "你是一个靠谱的{role}"),
    ("human", "{question}")
])

# 获取一个JsonOutputParser的实例
parser = JsonOutputParser()

# 写法1：
# prompt = chat_prompt_template.invoke(input={"role":"人工智能专家","question":"人工智能用英文怎么说？问题用q表示，答案用a表示，返回一个JSON格式的数据"})
#
# response = chat_model.invoke(prompt)
#
# json_result = parser.invoke(response)
# print(json_result)


# 写法2：
chain = chat_prompt_template | chat_model | parser
json_result1 = chain.invoke(
    input={"role": "人工智能专家", "question": "人工智能用英文怎么说？问题用q表示，答案用a表示，返回一个JSON格式的数据"})
print(json_result1)

{'q': 'What is the English word for artificial intelligence?', 'a': "Artificial intelligence is called 'artificial intelligence' in English."}


XMLOutputParser XML输出解析器的使用

In [10]:
chat_model = ChatOpenAI(model="qwen-turbo",
                        temperature=0.7,    # 1=创造力
                        max_tokens=100,)     # 最大数量

actor_query = "周星驰的简短电影记录"
response = chat_model.invoke(f"请生成{actor_query}，将影片附在<movie></movie>标签中")

print(type(response))
print(response.content)

<class 'langchain_core.messages.ai.AIMessage'>
<movie>
《唐伯虎点秋香》是周星驰于1993年自导自演的一部喜剧电影，讲述了唐伯虎为追求秋香而混入华府的故事。影片以无厘头风格为主，充满夸张的表演和搞笑桥段，是周星驰经典作品之一。
</movie>


In [11]:
from langchain_core.output_parsers.xml import XMLOutputParser

parser = XMLOutputParser()
print(parser.get_format_instructions())

The output should be formatted as a XML file.
1. Output should conform to the tags below.
2. If tags are not given, make them on your own.
3. Remember to always open and close all the tags.

As an example, for the tags ["foo", "bar", "baz"]:
1. String "<foo>
   <bar>
      <baz></baz>
   </bar>
</foo>" is a well-formatted instance of the schema.
2. String "<foo>
   <bar>
   </foo>" is a badly-formatted instance.
3. String "<foo>
   <tag>
   </tag>
</foo>" is a badly-formatted instance.

Here are the output tags:
```
None
```


In [12]:
# 1.导入相关包
from langchain_core.output_parsers import XMLOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 2. 初始化语言模型
chat_model = ChatOpenAI(model="qwen-turbo",
                        temperature=0.7,    # 1=创造力
                        max_tokens=100,)     # 最大数量

# 3.测试模型的xml解析效果
actor_query = "生成汤姆·汉克斯的简短电影记录,使用中文回复"

# 4.定义XMLOutputParser对象
parser = XMLOutputParser()

# 5. 生成提示词模板
prompt_template1 = PromptTemplate.from_template(
    template="用户的问题：{query}\n使用的格式：{format_instructions}"
)

prompt_template2 = prompt_template1.partial(format_instructions=parser.get_format_instructions())


response = chat_model.invoke(prompt_template2.invoke(input={"query": actor_query}))
print(response.content)


```xml
<filmography>
    <movie>
        <title>阿甘正传</title>
        <year>1994</year>
        <role>阿甘</role>
    </movie>
    <movie>
        <title>拯救大兵瑞恩</title>
        <year>1998</year>
        <role>詹姆斯·瑞恩</role>
    </movie>
    <movie>
        <title>绿里奇迹</title>
        <year>1999</year>
        <role>保尔·艾吉</role>
    </movie>
    <movie>
        <title>云图</title>
        <year>2012</year>
        <role>多个角色</role>
    </movie>
</filmography>
```


In [13]:
xml_result = parser.invoke(response)
print(xml_result)

{'filmography': [{'movie': [{'title': '阿甘正传'}, {'year': '1994'}, {'role': '阿甘'}]}, {'movie': [{'title': '拯救大兵瑞恩'}, {'year': '1998'}, {'role': '詹姆斯·瑞恩'}]}, {'movie': [{'title': '绿里奇迹'}, {'year': '1999'}, {'role': '保尔·艾吉'}]}, {'movie': [{'title': '云图'}, {'year': '2012'}, {'role': '多个角色'}]}]}


列表解析器 CommaSeparatedListOutputParser

In [14]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()

# 返回一些指令或模板，这些指令告诉系统如何解析或格式化输出数据
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

messages = "大象,猩猩,狮子"
result = output_parser.parse(messages)
print(result)
print(type(result))

Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`
['大象', '猩猩', '狮子']
<class 'list'>


In [20]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.output_parsers import CommaSeparatedListOutputParser

# 初始化语言模型
chat_model = ChatOpenAI(model="qwen-turbo",
                        temperature=0.7,    # 1=创造力
                        max_tokens=100,)     # 最大数量

# 创建解析器
output_parser = CommaSeparatedListOutputParser()

# 创建LangChain提示模板
chat_prompt = PromptTemplate.from_template(
    "生成5个关于{text}的列表.\n\n{format_instructions}",
    partial_variables={
    "format_instructions": output_parser.get_format_instructions()
    })

# 提示模板与输出解析器传递输出
# chat_prompt = chat_prompt.partial(format_instructions=output_parser.get_format_instructions())

# 将提示和模型合并以进行调用
chain = chat_prompt | chat_model | output_parser
res = chain.invoke({"text": "电影"})
print(res)
print(type(res))

['星际穿越', '肖申克的救赎', '阿甘正传', '盗梦空间', '泰坦尼克号']
<class 'list'>


日期解析器 DatetimeOutputParser

In [19]:
from langchain.output_parsers import DatetimeOutputParser

output_parser = DatetimeOutputParser()

format_instructions = output_parser.get_format_instructions()
print(format_instructions)

Write a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.

Examples: 1871-02-03T01:55:36.792831Z, 0525-06-11T23:29:04.907217Z, 0296-03-27T12:34:15.110586Z

Return ONLY this string, no other words!


In [18]:
from langchain_openai import ChatOpenAI
from langchain.prompts.chat import HumanMessagePromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import DatetimeOutputParser

chat_model = ChatOpenAI(model="qwen-turbo",
                        temperature=0.7,    # 1=创造力
                        max_tokens=100,)     # 最大数量


chat_prompt = ChatPromptTemplate.from_messages([
    ("system","{format_instructions}"),
    ("human", "{request}")
])

output_parser = DatetimeOutputParser()

# 方式1：
# model_request = chat_prompt.format_messages(
#     request="中华人民共和国是什么时候成立的",
#     format_instructions=output_parser.get_format_instructions()
# )

# response = chat_model.invoke(model_request)
# result = output_parser.invoke(response)
# print(result)
# print(type(result))

# 方式2：
chain = chat_prompt | chat_model | output_parser
resp = chain.invoke({"request":"中华人民共和国是什么时候成立的",
                     "format_instructions":output_parser.get_format_instructions()})
print(resp)
print(type(resp))

1949-10-01 00:00:00
<class 'datetime.datetime'>
